In [ ]:
import numpy as np
import pandas as pd

# Load tsbs_placement.csv to get available locations
tsbs_data = pd.read_csv('tsbs_placement_imp.csv')

# Define parameters
S = 50  # Total population size
U = 4   # Total number of child UAVs
min_altitude = 300
max_altitude = 800


# Initialize population
population = []

# Generate S chromosomes
for s in range(S):
    # Initialize a chromosome representing location details of child UAVs
    chromosome = np.zeros((U, 3))  # U rows, 3 columns (latitude, longitude, altitude)

    # Generate random location details for each child UAV
    for i in range(U):
        # Generate random latitude and longitude within the range of tsbs_placement
        lat = np.random.uniform(tsbs_data['latitude'].min(), tsbs_data['latitude'].max())
        lon = np.random.uniform(tsbs_data['longitude'].min(), tsbs_data['longitude'].max())

        # Random altitude
        h = np.random.randint(min_altitude, max_altitude + 1)

        # Assign location details to chromosome
        chromosome[i] = [lat, lon, h]

    # Add chromosome to the population
    population.append(chromosome)

# Convert population to NumPy array
population = np.array(population)

# Save each chromosome separately
for s, chromosome in enumerate(population):
    # Initialize DataFrame for the chromosome
    chromosome_data = pd.DataFrame({
        'latitude': chromosome[:, 0],
        'longitude': chromosome[:, 1],
        'altitude': chromosome[:, 2]
    })

    # Define the filename for the chromosome
    filename = f"chromosome_{s+1}.csv"

    # Save chromosome to CSV file
    chromosome_data.to_csv(filename, index=False)

print("Chromosome files generated successfully.")


Chromosome files generated successfully.


In [ ]:
import math
import csv
import pandas as pd
import numpy as np
from scipy.stats import nakagami

# Constants
frequency = 2e9
c = 3e8
a = 9.16
b = 0.16
attenuation_LoS = 1.0
attenuation_NLoS = 20

# Load TSBS and child UAV locations from CSV files
def load_tsbs_locations(tsbs_csv):
    tsbs_df = pd.read_csv(tsbs_csv)
    tsbs_df['tsbs_index'] = range(1, len(tsbs_df) + 1)
    return tsbs_df[['tsbs_index', 'latitude', 'longitude']]



# Path loss calculation
def path_loss(distance, elevation_angle):
    wavelength = c / frequency

    # Check if the distance is zero (same location), set LoS probability to 1
    if distance == 0:
        PLoS = 1
    else:
        # Ensure that the argument of math.exp is within a reasonable range
        arg = -b * (elevation_angle - a)

        PLoS = 1 / (1 + a * math.exp(arg))

    PNLoS = 1 - PLoS

    # Ensure the argument of math.log10 is never zero
    if distance == 0:
        path_loss_dB = 0
    else:
        path_loss_dB = (20 * math.log10(4 * math.pi * distance / wavelength)) + PLoS * attenuation_LoS  + PNLoS * attenuation_NLoS

    return path_loss_dB, PLoS, PNLoS

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in km

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    distance = R * c  # Distance in kilometers
    return distance * 1000  # Convert to meters

# Calculate distance between TSBS and child-UAV
def calculate_distance(tsbs_latitude, tsbs_longitude, child_uav_latitude, child_uav_longitude):
    return haversine(tsbs_latitude, tsbs_longitude, child_uav_latitude, child_uav_longitude)
# Calculate elevation angle between TSBS and child-UAV
def calculate_elevation_angle(distance, altitude):
    return math.atan(altitude / distance)

# Calculate transmit power in dBm from watts
def watts_to_dbm(watts):
    return 10 * math.log10(watts)

# Calculate fading in dB
def calculate_fading(PLoS, PNLoS, m):
    z0 = np.abs(nakagami.rvs(m, size=1))
    z1 = np.abs(nakagami.rvs(m, size=1))
    return PLoS * z0 + PNLoS * z1

# Calculate channel gain in dB
def calculate_channel_gain():
    N = int(1e6)
    fading_coefficients = np.abs((1/np.sqrt(2)) * (np.random.randn(1, N) + 1j*np.random.randn(1, N)))
    fading_coefficients= fading_coefficients.mean()
    return 10*math.log10(fading_coefficients)

# Calculate received power in dBm from watts
def received_power_dbm(optimal_power, path_loss, PLoS, PNLoS, m):
    fading = calculate_fading(PLoS, PNLoS, m)
    return 10 * np.log10(optimal_power) + fading - path_loss

# Calculate SINR
def calculate_sinr(received_power, interference_sum, noise_power_dbm):
    return received_power / (noise_power_dbm + interference_sum)

# Calculate interference sum
def calculate_interference_sum(channel_gain_df):
    interference_sum = channel_gain_df.groupby(['tsbs_index'])['received_power_dB'].transform('sum') - channel_gain_df['received_power_dB']
    return interference_sum

# Calculate bandwidth
def calculate_bandwidth(data_rate, sinr):
    return data_rate / np.log2(1 + sinr)

# Calculate spectral efficiency
def calculate_spectral_efficiency(data_rate, bandwidth):
    return data_rate / bandwidth

# Function to calculate all parameters and optimize allocation for each chromosome
def calculate_parameters_and_optimize(chromosome):
    # Load child UAV locations from chromosome file
    child_uav_locations = pd.read_csv(chromosome)

    # Add a unique uav_index to the child UAV locations
    child_uav_locations['uav_index'] = range(1, len(child_uav_locations) + 1)

    # Load TSBS locations
    tsbs_csv = "tsbs_placement_imp.csv"
    tsbs_locations = load_tsbs_locations(tsbs_csv)

    # Generate random data rate values
    data_rates = np.random.choice([20, 40, 60, 80, 100], size=len(tsbs_locations) * len(child_uav_locations))

    # Calculate path loss, fading, and channel gain for each TSBS-child UAV pair and assign data rate
    channel_gain_data = []
    for idx_tsbs, tsbs_row in tsbs_locations.iterrows():
        for idx_uav, child_uav_row in child_uav_locations.iterrows():
            distance = calculate_distance(tsbs_row['latitude'], tsbs_row['longitude'], child_uav_row['latitude'], child_uav_row['longitude'])
            elevation_angle = calculate_elevation_angle(distance, child_uav_row['altitude'])
            path_loss_dB, PLoS, PNLoS = path_loss(distance, elevation_angle)

            # Calculate fading in dB
            m = 1  # Shape parameter (m = 1 for Rayleigh fading)
            fading_db = watts_to_dbm(calculate_fading(PLoS, PNLoS, m))

            transmit_power_dbm = watts_to_dbm(1.3)  # Transmit power of 1.3 watts converted to dBm
            channel_gain = calculate_channel_gain()
            data_rate = data_rates[idx_tsbs * len(child_uav_locations) + idx_uav]
            channel_gain_data.append((tsbs_row['tsbs_index'], child_uav_row['uav_index'], path_loss_dB, PLoS, PNLoS, channel_gain, data_rate))

    # Save results to DataFrame
    channel_gain_df = pd.DataFrame(channel_gain_data, columns=['tsbs_index', 'uav_index', 'path_loss', 'PLoS', 'PNLoS', 'channel_gain', 'data_rate'])

    # Calculate received power in dBm
    m = 1  # Shape parameter (m = 1 for Rayleigh fading)
    channel_gain_df['received_power_dB'] = received_power_dbm(1.3, channel_gain_df['path_loss'], channel_gain_df['PLoS'], channel_gain_df['PNLoS'], m)

    # Calculate SINR
    noise_power_dbm = -125  # Noise power in dBm
    channel_gain_df['interference_sum'] = calculate_interference_sum(channel_gain_df)
    channel_gain_df['SINR'] = calculate_sinr(channel_gain_df['received_power_dB'], channel_gain_df['interference_sum'], noise_power_dbm)

    # Calculate bandwidth
    channel_gain_df['bandwidth'] = calculate_bandwidth(channel_gain_df['data_rate'], channel_gain_df['SINR'])

    # Calculate spectral efficiency
    channel_gain_df['spectral_efficiency'] = calculate_spectral_efficiency(channel_gain_df['data_rate'], channel_gain_df['bandwidth'])

    channel_gain_df.to_csv("path_loss.csv", index=False)


    # Load the data from the CSV file
    df = pd.read_csv("path_loss.csv")

    # Find unique 'sbs_index' and 'uav_index'
    unique_tsbs_index = df['tsbs_index'].unique()
    unique_uav_index = df['uav_index'].unique()

    # Initialize the matrix A with zeros
    A = pd.DataFrame(0, index=unique_tsbs_index, columns=unique_uav_index)

    # Minimum SINR threshold
    min_sinr = -10

    # Iterate over each unique 'sbs_index'
    for tsbs_index in unique_tsbs_index:
        # Filter rows for the current 'sbs_index'
        tsbs_data = df[df['tsbs_index'] == tsbs_index]

        # Find the child-UAV providing maximum SINR
        max_sinr_row = tsbs_data.loc[tsbs_data['SINR'].idxmax()]
        max_sinr_uav_index = max_sinr_row['uav_index']
        max_sinr_value = max_sinr_row['SINR']

        # Check if SINR exceeds the minimum SINR threshold
        if max_sinr_value > min_sinr:
            A.loc[tsbs_index, max_sinr_uav_index] = 1

    # Initialize the matrix A_updated with zeros
    A_updated = A.copy()

    # Initialize counters
    num_links_updated = 0
    C_bandwidth = 0
    max_bandwidth=600
    max_links = 7

    # Iterate over each child-UAV
    for uav_index in unique_uav_index:
        existing_links = A_updated.index[A_updated[uav_index] == 1]

        # Get data for the current child-UAV
        uav_data = df[df['uav_index'] == uav_index]

        # Initialize counters
        C_bandwidth = 0
        num_links_updated = 0

        # Iterate through TSBSs until maximum resources are utilized
        for tsbs_index in uav_data['tsbs_index'].unique():
            # Filter data for the current tsbs_index and sort by spectral efficiency
            tsbs_data = uav_data[uav_data['tsbs_index'] == tsbs_index]
            tsbs_data = tsbs_data.sort_values(by='spectral_efficiency', ascending=False)

            # Iterate through sorted data
            for _, row in tsbs_data.iterrows():
                bandwidth = row['bandwidth']

                # Check if adding the bandwidth exceeds maximum allowed bandwidth or maximum number of links
                if C_bandwidth + bandwidth <= max_bandwidth and num_links_updated < max_links:
                    # Update matrix A and counters
                    if tsbs_index in existing_links:
                        A_updated.loc[tsbs_index, uav_index] = 1
                        C_bandwidth += bandwidth

                else:
                    # If adding bandwidth exceeds the maximum allowed, break the loop
                    A_updated.loc[tsbs_index, uav_index] = 0
                    break

    # Ensure each column (UAV) has no more than 7 ones (1s)
    for uav_index in unique_uav_index:
        column_sum = A_updated[uav_index].sum()
        if column_sum > max_links:
            # Find indices of extra ones and set them to zero
            extra_indices = A_updated.index[A_updated[uav_index] == 1][max_links:]
            A_updated.loc[extra_indices, uav_index] = 0


    # Calculate sum rate F_s
    F_s = 0
    for tsbs_index in unique_tsbs_index:
        for uav_index in unique_uav_index:
            if A_updated.loc[tsbs_index, uav_index] == 1:
                data_rate = df[(df['tsbs_index'] == tsbs_index) & (df['uav_index'] == uav_index)]['data_rate'].values[0]
                F_s += data_rate

    C_prime = A_updated.sum().sum()

    # Initialize Cb as the cumulative bandwidth used by associated TSBSs
    C_b = (A_updated * df['bandwidth']).sum().sum()

    # Initialize Fs as total sum-rate of associated TSBSs
    Fs = F_s

    # Initialize RB (required bandwidth) as 1.6 Gbps
    RB = 1.6 * 1000  # Convert Gbps to Mbps

    # Continue looping until Fs is less than or equal to RB
    while Fs > RB:
        # Select child-UAV with max associated TSBSs
        max_tsbs_uav = A_updated.sum(axis=0).idxmax()

        # Select TSBS with minimum data rate demand
        min_data_rate_tsbs = df.loc[df['uav_index'] == max_tsbs_uav].sort_values(by='data_rate').iloc[0]
        min_data_rate_tsbs_index = min_data_rate_tsbs['tsbs_index']
        min_data_rate_data_rate = min_data_rate_tsbs['data_rate']

        # De-associate the selected pair and update matrix A, decrement C', update Fs, and decrement Cb
        A_updated.loc[min_data_rate_tsbs_index, max_tsbs_uav] = 0
        C_prime -= 1
        Fs -= min_data_rate_data_rate
        C_b -= min_data_rate_tsbs['bandwidth']


    #A_updated = pd.DataFrame(index=tsbs_locations['tsbs_index'], columns=child_uav_locations['uav_index'])
    num_ones = A_updated.values.sum()



    # Return the number of chromosome, sum rate F_s, and number of 1s in the A_updated matrix
    return chromosome, F_s, num_ones

# Main loop to process each chromosome
results = []
for chromosome_num in range(1,51):
    chromosome_file = f"chromosome_{chromosome_num}.csv"
    chromosome, F_s, num_ones = calculate_parameters_and_optimize(chromosome_file)
    results.append((chromosome, F_s, num_ones))

# Save results to a CSV file
results_df = pd.DataFrame(results, columns=['Chromosome', 'Sum_Rate_F_s', 'Number_of_Ones_in_A_updated'])
results_df.to_csv("optimization_results.csv", index=False)

print("Optimization completed for all chromosomes. Results saved to optimization_results.csv.")


<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will erro

Optimization completed for all chromosomes. Results saved to optimization_results.csv.


In [ ]:
import pandas as pd

# Load optimization_results.csv
results_df = pd.read_csv("optimization_results.csv")

# Sort the DataFrame in descending order of Sum_Rate_F_s and Number_of_Ones_in_A_updated
sorted_results_df = results_df.sort_values(by=['Sum_Rate_F_s', 'Number_of_Ones_in_A_updated'], ascending=False)

# Save the sorted DataFrame to a new CSV file
sorted_results_df.to_csv("optimization_results.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np

# Load the optimization_results.csv file
results_df = pd.read_csv("optimization_results.csv")

# Define fitness function (you need to define your own fitness function)
def fitness_function(row):
    # Example fitness function: Sum of sum rate and number of ones
    return row['Sum_Rate_F_s']

# Calculate fitness values
results_df['Fitness'] = results_df.apply(fitness_function, axis=1)

# Sort the DataFrame by fitness in descending order
sorted_results_df = results_df.sort_values(by='Fitness', ascending=False)

# Calculate the number of elite parents (40% of total chromosomes)
num_elite_parents = int(0.4 * len(sorted_results_df))

# Select elite parents
elite_parents = sorted_results_df.head(num_elite_parents)
elite_parents=elite_parents['Chromosome']

# Perform roulette wheel selection for the remaining chromosomes
remaining_chromosomes = sorted_results_df.tail(len(sorted_results_df) - num_elite_parents)

# Calculate probabilities for RWS
total_fitness = remaining_chromosomes['Fitness'].sum()
remaining_chromosomes['Probability'] = remaining_chromosomes['Fitness'] / total_fitness

# Perform RWS to select remaining chromosomes
selected_indices = np.random.choice(remaining_chromosomes.index, size=len(remaining_chromosomes), replace=True, p=remaining_chromosomes['Probability'])
selected_chromosomes = remaining_chromosomes.loc[selected_indices]
selected_chromosomes=selected_chromosomes['Chromosome']


<ipython-input-39-ca4abe19e35e>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_chromosomes['Probability'] = remaining_chromosomes['Fitness'] / total_fitness


In [ ]:
import pandas as pd
import numpy as np
import random

# Mutation parameters
mutation_rate = 0.09
gmtd_value = 4

# Function to apply mutation to a single chromosome
def apply_mutation(chromosome_file):
    # Load the chromosome file
    df = pd.read_csv(chromosome_file)

    # Get the total number of child UAVs in the chromosome
    num_child_uavs = len(df)

    # Calculate the number of child UAVs to mutate based on mutation rate
    num_to_mutate = int(round(mutation_rate * num_child_uavs))

    # Select random child UAVs to mutate
    indices_to_mutate = random.sample(range(num_child_uavs), num_to_mutate)

    # Iterate over selected indices and apply mutation
    for idx in indices_to_mutate:
        # Coin toss to decide whether to add or subtract gmtd_value
        if random.random() < 0.5:
            # Add gmtd_value
            df.loc[idx, 'latitude'] += gmtd_value
        else:
            # Subtract gmtd_value
            df.loc[idx, 'latitude'] -= gmtd_value

        # Similarly mutate Y-coordinate and altitude
        if random.random() < 0.5:
            df.loc[idx, 'longitude'] += gmtd_value
        else:
            df.loc[idx, 'longitude'] -= gmtd_value

        if random.random() < 0.5:
            df.loc[idx, 'altitude'] += gmtd_value
        else:
            df.loc[idx, 'altitude'] -= gmtd_value

    # Save the mutated chromosome to the same file
    df.to_csv(chromosome_file, index=False)

    return chromosome_file

# List of selected chromosome files obtained from roulette wheel selection
selected_chromosomes
# Apply mutation to each selected chromosome file
mutated_files = []
for chromosome_file in selected_chromosomes:
    mutated_file = apply_mutation(chromosome_file)
    mutated_files.append(mutated_file)

print("Mutation applied to selected chromosomes.")


Mutation applied to selected chromosomes.


In [ ]:
selected_chromosomes_files=pd.concat([elite_parents, selected_chromosomes])
selected_chromosomes_files.head()

0    chromosome_31.csv
1    chromosome_13.csv
2    chromosome_32.csv
3    chromosome_26.csv
4     chromosome_4.csv
Name: Chromosome, dtype: object

In [ ]:
results = []
for chromosome_file in selected_chromosomes_files:
    chromosome, F_s, num_ones = calculate_parameters_and_optimize(chromosome_file)
    results.append((chromosome, F_s, num_ones))

# Save results to a CSV file
results_df = pd.DataFrame(results, columns=['Chromosome', 'Sum_Rate_F_s', 'Number_of_Ones_in_A_updated'])
results_df.to_csv("optimization_results.csv", index=False)

<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will erro

In [ ]:
import pandas as pd
import numpy as np
import random

# Function to apply mutation to a single chromosome
def apply_mutation(chromosome_file):
    # Load the chromosome file
    df = pd.read_csv(chromosome_file)

    # Get the total number of child UAVs in the chromosome
    num_child_uavs = len(df)

    # Calculate the number of child UAVs to mutate based on mutation rate
    num_to_mutate = int(round(mutation_rate * num_child_uavs))

    # Select random child UAVs to mutate
    indices_to_mutate = random.sample(range(num_child_uavs), num_to_mutate)

    # Iterate over selected indices and apply mutation
    for idx in indices_to_mutate:
        # Coin toss to decide whether to add or subtract gmtd_value
        if random.random() < 0.5:
            # Add gmtd_value
            df.loc[idx, 'latitude'] += gmtd_value
        else:
            # Subtract gmtd_value
            df.loc[idx, 'latitude'] -= gmtd_value

        # Similarly mutate Y-coordinate and altitude
        if random.random() < 0.5:
            df.loc[idx, 'longitude'] += gmtd_value
        else:
            df.loc[idx, 'longitude'] -= gmtd_value

        if random.random() < 0.5:
            df.loc[idx, 'altitude'] += gmtd_value
        else:
            df.loc[idx, 'altitude'] -= gmtd_value

    # Save the mutated chromosome to the same file
    df.to_csv(chromosome_file, index=False)

    return chromosome_file

# Perform iteration
top_sum_rate = None
unchanged_iterations = 0
for _ in range(10):  # 50 iterations
    # Load optimization_results.csv
    results_df = pd.read_csv("optimization_results.csv")

    # Sort the DataFrame in descending order of Sum_Rate_F_s and Number_of_Ones_in_A_updated
    sorted_results_df = results_df.sort_values(by=['Sum_Rate_F_s', 'Number_of_Ones_in_A_updated'], ascending=False)

    # Save the sorted DataFrame to a new CSV file
    sorted_results_df.to_csv("optimization_results.csv", index=False)

    # Load the optimization_results.csv file
    results_df = pd.read_csv("optimization_results.csv")

    # Define fitness function
    def fitness_function(row):
        return row['Sum_Rate_F_s']

    # Calculate fitness values
    results_df['Fitness'] = results_df.apply(fitness_function, axis=1)

    # Sort the DataFrame by fitness in descending order
    sorted_results_df = results_df.sort_values(by='Fitness', ascending=False)

    # Calculate the number of elite parents (40% of total chromosomes)
    num_elite_parents = int(0.4 * len(sorted_results_df))

    # Select elite parents
    elite_parents = sorted_results_df.head(num_elite_parents)
    elite_parents = elite_parents['Chromosome']

    # Perform roulette wheel selection for the remaining chromosomes
    remaining_chromosomes = sorted_results_df.tail(len(sorted_results_df) - num_elite_parents)

    # Calculate probabilities for RWS
    total_fitness = remaining_chromosomes['Fitness'].sum()
    remaining_chromosomes['Probability'] = remaining_chromosomes['Fitness'] / total_fitness

    # Perform RWS to select remaining chromosomes
    selected_indices = np.random.choice(remaining_chromosomes.index, size=len(remaining_chromosomes), replace=True, p=remaining_chromosomes['Probability'])
    selected_chromosomes = remaining_chromosomes.loc[selected_indices]
    selected_chromosomes = selected_chromosomes['Chromosome']

    # Combine elite parents and selected chromosomes
    selected_chromosomes_files = pd.concat([elite_parents, selected_chromosomes])

    # Apply mutation to each selected chromosome file
    mutated_files = []
    for chromosome_file in selected_chromosomes_files:
        mutated_file = apply_mutation(chromosome_file)
        mutated_files.append(mutated_file)

    print(f"Iteration {_+1}: Mutation applied to selected chromosomes.")

    # Run optimization on mutated chromosomes and update optimization_results.csv
    results = []
    for chromosome_file in selected_chromosomes_files:
        chromosome, F_s, num_ones = calculate_parameters_and_optimize(chromosome_file)
        results.append((chromosome, F_s, num_ones))

    # Save results to a CSV file
    results_df = pd.DataFrame(results, columns=['Chromosome', 'Sum_Rate_F_s', 'Number_of_Ones_in_A_updated'])
    results_df.to_csv("optimization_results.csv", index=False)

    # Check if the top sum rate has changed
    current_top_sum_rate = results_df['Sum_Rate_F_s'].max()
    if top_sum_rate is None:
        top_sum_rate = current_top_sum_rate
    elif current_top_sum_rate == top_sum_rate:
        unchanged_iterations += 1
    else:
        top_sum_rate = current_top_sum_rate
        unchanged_iterations = 0

    # If the top sum rate has remained unchanged for 5 iterations, break the loop
    if unchanged_iterations == 5:
        print("Top sum rate has remained unchanged for 5 iterations. Exiting...")
        break

# Print the final top chromosome file and its parameters
final_top_chromosome = results_df.loc[results_df['Sum_Rate_F_s'].idxmax()]
print("Final Top Chromosome:")
print(final_top_chromosome)

# Read and print the final top chromosome file
final_top_chromosome_file = final_top_chromosome['Chromosome']
final_top_chromosome_data = pd.read_csv(final_top_chromosome_file)
final_top_chromosome_file = final_top_chromosome[['Chromosome','Sum_Rate_F_s']]
print("\nFinal Top Chromosome Data:")
print(final_top_chromosome_data)
max_bandwidth=600
final_top_chromosome_file['bandwidth'] = max_bandwidth

# Create a CSV file containing the chromosome, its sum rate, and the maximum bandwidth
final_top_chromosome_file.to_csv("final_top_chromosome_file.csv", index=False)

<ipython-input-43-96ee0ea1060c>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_chromosomes['Probability'] = remaining_chromosomes['Fitness'] / total_fitness
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)


Iteration 1: Mutation applied to selected chromosomes.


<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will erro

Iteration 2: Mutation applied to selected chromosomes.


<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will erro

...

Iteration 9: Mutation applied to selected chromosomes.


<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will erro

Iteration 10: Mutation applied to selected chromosomes.


<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 10 * math.log10(watts)
<ipython-input-37-0f67aec49538>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will erro

Final Top Chromosome:
Chromosome                     chromosome_31.csv
Sum_Rate_F_s                                 760
Number_of_Ones_in_A_updated                   13
Name: 0, dtype: object

Final Top Chromosome Data:
    latitude  longitude  altitude
0  40.754475 -73.977038     527.0
1  40.734952 -73.963795     608.0
2  40.730949 -73.982238     522.0
3  40.755872 -73.989822     489.0
